In [1]:
from smach_utils2 import *

[ WARN] [1685049531.269814170]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1685049531.275826733]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1685049531.275898263]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1685049531.277391480]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1685049531.277464711]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1685049531.568547860, 3628.220000000]: Kinematics solver doesn't suppo

In [ ]:
tfBuffer = tf2.Buffer()

listener = tf2.TransformListener(tfBuffer)
b_tf=tf2.TransformBroadcaster()
b_st=tf2.StaticTransformBroadcaster()


In [ ]:
def write_tf(pose, q, child_frame , parent_frame='map',time=0):
    t= TransformStamped()
    t.header.stamp = rospy.Time.now()
    t.header.frame_id =parent_frame
    t.child_frame_id =  child_frame
    t.transform.translation.x = pose[0]
    t.transform.translation.y = pose[1]
    t.transform.translation.z = pose[2]
    #q = tf.transformations.quaternion_from_euler(eu[0], eu[1], eu[2])
    t.transform.rotation.x = q[0]
    t.transform.rotation.y = q[1]
    t.transform.rotation.z = q[2]
    t.transform.rotation.w = q[3]
    return t
def read_tf(t):
    pose=np.asarray((
        t.transform.translation.x,
        t.transform.translation.y,
        t.transform.translation.z
        ))
    quat=np.asarray((
        t.transform.rotation.x,
        t.transform.rotation.y,
        t.transform.rotation.z,
        t.transform.rotation.w
        ))
    
    return pose, quat


In [ ]:
res=segmentation_server.call()

In [ ]:
res

In [ ]:
poses=np.asarray(res.poses.data)

poses=poses.reshape((int(len(poses)/3) ,3     )      )

In [ ]:
head.set_named_target('neutral')

In [ ]:
#t=write_tf(res.poses.data,(0,0,0,1),'Target',"head_rgbd_sensor_depth_frame",0)
t=write_tf(poses[0,:],(0,0,0,1),'Target_tmp',"head_rgbd_sensor_depth_frame",0) ## CHOOSE A TARGET 

b_tf.sendTransform(t)
rospy.sleep(0.2)
tt=tfBuffer.lookup_transform('map','Target_tmp',rospy.Time(0))
rospy.sleep(0.2)
tt
pose,quat= read_tf(tt)
t=write_tf(pose,(0,0,0,1),'Target')
b_st.sendTransform(t)


In [ ]:
head = moveit_commander.MoveGroupCommander('head')
#gripper =  moveit_commander.MoveGroupCommander('gripper')
#arm=moveit_commander.MoveGroupCommander('arm')

In [ ]:
#whole_body=moveit_commander.MoveGroupCommander('whole_body')

In [ ]:
clear_octo_client()
arm_grasp_from_above = [0.3, -2.0, 0.0, -0.9939144210462025, -0.17365421548386273, 0.0]
wb_v=whole_body.get_current_joint_values()
wb_v[3:]=arm_grasp_from_above
succ = whole_body.go(wb_v)

In [ ]:
head.set_named_target('neutral')
head.go()
clear_octo_client()


In [ ]:
#
t=tfBuffer.lookup_transform('map','Target',rospy.Time(0))
pose , quat =  read_tf(t)
pose

In [ ]:
wb_gp=whole_body.get_current_pose()
wb_gp.pose.position.x= pose[0]
wb_gp.pose.position.y= pose[1]#
wb_gp.pose.position.z= pose[2]+0.3   ##Pre Grasp from above


In [ ]:
whole_body.set_pose_target(wb_gp)
plan=whole_body.plan()
if plan[0]:
    succ=whole_body.go()
else:
    print('No plan')

In [ ]:
succ

In [ ]:
t=tfBuffer.lookup_transform('hand_palm_link', 'Target',rospy.Time())

In [ ]:
pose,_=read_tf(t)
pose

In [ ]:
goal_p=whole_body.get_current_pose()
goal_p.pose.position.x=0.1
goal_p.pose.position.y=pose[1]
goal_p.pose.position.z=pose[2]+0.2
#goal_p.pose.orientation.w=quat[0]
#goal_p.pose.orientation.x=quat[1]
#goal_p.pose.orientation.y=quat[2]
#goal_p.pose.orientation.z=quat[3]



In [ ]:
whole_body.set_pose_target(goal_p)

In [ ]:
whole_body.plan()

In [ ]:
goal_p=whole_body.get_current_pose()

In [ ]:
whole_body.set_pose_target(goal_p)
whole_body.plan()

In [ ]:
whole_body.go()

In [ ]:
res.poses.data

In [ ]:
t=tfBuffer.lookup_transform('map','Target',rospy.Time())

In [ ]:
pose,quat=read_tf(t)

In [ ]:
goal_pose= Pose()

In [ ]:
goal_pose.position.x=pose[0]
goal_pose.position.y=pose[1]
goal_pose.position.z=pose[2]

In [ ]:
pose,goal_pose

In [ ]:
whole_body.get_current_pose()

In [ ]:
wb=whole_body.get_current_joint_values()

In [ ]:
wb[4]=0.1

In [ ]:
whole_body.go(wb)

In [ ]:
pose=whole_body.get_current_pose()

In [ ]:
type(pose.pose.orientation)

In [ ]:
tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))

In [ ]:
tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))

In [ ]:
gp=whole_body.get_current_pose()

In [ ]:

gp.pose.orientation
pose=gp
eu=tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))
#eu[1]=0
quat=tf.transformations.quaternion_from_euler(eu[0],eu[1],eu[2])

In [ ]:
eu=tf.transformations.euler_from_quaternion((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))

In [ ]:
tfBuffer.lookup_transform('hand_palm_link','New:static_Target',rospy.Time(0))

In [ ]:
#eu[1]=0
quat=tf.transformations.quaternion_from_euler(eu[0],eu[1],eu[2])
rot_quat=tf.transformations.quaternion_from_euler(0,0.5*np.pi,0.0)

In [ ]:
wb_gp=whole_body.get_current_pose()

In [ ]:
eu=tf.transformations.euler_from_quaternion((wb_gp.pose.orientation.w,wb_gp.pose.orientation.x,wb_gp.pose.orientation.y,wb_gp.pose.orientation.z))
quat=tf.transformations.quaternion_from_euler(eu[0],eu[1],eu[2])
rot_quat=tf.transformations.quaternion_from_euler(0,0.5*np.pi,0.0)
rotated=tf.transformations.quaternion_multiply(quat,rot_quat)



In [ ]:
wb_gp.pose.position.x-=0.1
wb_gp.pose.position.y+=0.1
wb_gp.pose.position.z-=0.1


In [ ]:
wb_gp.pose.orientation.w=rotated[0]
wb_gp.pose.orientation.x=rotated[1]
wb_gp.pose.orientation.y=rotated[2]
wb_gp.pose.orientation.z=rotated[3]

In [ ]:
whole_body.set_pose_target(wb_gp)

In [ ]:

plan=whole_body.plan()
plan

In [ ]:
if plan[0]:
    whole_body.go()

In [ ]:
whole_body.go()

In [ ]:
arm_grasp_from_above = [0.19263830140116414, -2.2668981568652917, -0.007358947463759424, -0.9939144210462025, -0.17365421548386273, 0.0]
wb_v=whole_body.get_current_joint_values()
wb_v[3:]=arm_grasp_from_above
succ = whole_body.go(wb_v)

In [ ]:

t=tfBuffer.lookup_transform('New:static_Target','hand_palm_link',rospy.Time(0))
pose , quat =  read_tf(t)
pose,quat
np.save('above_quat.npy',quat)

In [ ]:
t=tfBuffer.lookup_transform('map','Target',rospy.Time(0))
pose , quat =  read_tf(t)
pose,quat

In [ ]:
wb_gp=whole_body.get_current_pose()
wb_gp.pose.position.x= pose[0]
wb_gp.pose.position.y= pose[1]
wb_gp.pose.position.z= pose[2]+0.2


In [ ]:
whole_body.set_pose_target(wb_gp)
plan=whole_body.plan()
plan

In [ ]:

whole_body.go()

In [ ]:
wb_v=whole_body.get_current_joint_values()


In [ ]:
gripper = GRIPPER()

In [ ]:
gripper.close()

In [ ]:
gripper =  moveit_commander.MoveGroupCommander('gripper')

In [ ]:

|
tf_man.pub_static_tf()

In [ ]:
head.set_joint_values([0.0, -0.77])


In [ ]:
print ('rdy')

In [ ]:
brazo.set_named_target('go')
head.set_joint_values([0.0, -0.77])
res=segmentation_server.call()
if len(res.poses.data)==0: print( 'failed')
else:

    poses=np.asarray(res.poses.data)
    poses=poses.reshape((int(len(poses)/3) ,3     )      )  





    tf_man.pub_static_tf(pos=poses[0,:], point_name='target', ref='head_rgbd_sensor_rgb_frame')## which object to choose   #TODO
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(point_name='target', new_frame='map')
    rospy.sleep(0.3)
    pose, _ =tf_man.getTF('target')
    tf_man.pub_static_tf(pos=pose, point_name='Target')## which object to choose   #TODO


    head.set_named_target('neutral')

In [ ]:
poses=np.asarray(res.poses.data)
poses=poses.reshape((int(len(poses)/3) ,3     )      )  





tf_man.pub_static_tf(pos=poses[0,:], point_name='target', ref='head_rgbd_sensor_rgb_frame')## which object to choose   #TODO

In [ ]:
tf_man.change_ref_frame_tf(point_name='target', new_frame='map')


In [ ]:
pose, _ =tf_man.getTF('target')
tf_man.pub_static_tf(pos=pose, point_name='Target')## which object to choose   #TODO


In [ ]:
head.set_named_target('neutral')

In [ ]:
 clear_octo_client()
#############################10 * 2 *np.pi / 360 ####pi /2
arm_grasp_from_above = [0.2, -1.57, -0.13 , -1.57, 0.0, 0.0]
wb_v=whole_body.get_current_joint_values()
wb_v[3:]=arm_grasp_from_above
succ = whole_body.go(wb_v)   

In [ ]:
clear_octo_client()
pose, quat=tf_man.getTF('Target')

wb_gp=whole_body.get_current_pose()
wb_gp.pose.position.x= pose[0]
wb_gp.pose.position.y= pose[1]
wb_gp.pose.position.z= pose[2]+0.2

whole_body.set_pose_target(wb_gp)
plan=whole_body.plan()

In [ ]:
pos,rot=tf_man.getTF(target_frame=,ref)
pos

In [ ]:
from smach_utils2 import *

In [ ]:
from smach_utils2 import *


In [ ]:
wb_gp=whole_body.get_current_pose()            
wb_gp.header.frame_id="Target"
wb_gp.pose.position.x=0.02669
wb_gp.pose.position.y=0.045
wb_gp.pose.position.z= 0.033
wb_gp.pose.orientation.w=-0.990
wb_gp.pose.orientation.x= 0.0246
wb_gp.pose.orientation.y= -0.0085
wb_gp.pose.orientation.z= 0.1345



In [ ]:
aaa

In [ ]:
from smach_ros import ActionServerWrapper

In [ ]:
res = speech_recog_server()

In [ ]:
res.data

#msg = rospy.wait_for_message('/speech_recognition/final_result', String, 10)
#msg

In [ ]:
if len(res.data.split(' '))>1:print(res.data.split(' ')[-1])    ### Number of words


In [ ]:
from ros_whisper_vosk.srv import SetGrammarVosk
set_grammar = rospy.ServiceProxy('set_grammar_vosk', SetGrammarVosk)

In [ ]:
set_grammar = rospy.ServiceProxy('set_grammar_vosk', SetGrammarVosk)

In [ ]:
set_grammar(['pete','ana','jack', 'michael', ' my name is' , 'i am','george','mary','ruben','oscar','yolo','mitzi'])  ##PRESET NAMES

In [ ]:
msg = rospy.wait_for_message('/speech_recognition/final_result', String, 10)
msg

In [ ]:
msg = rospy.wait_for_message('/speech_recognition/final_result', String, 10)
#msg.data
if len(msg.data.split(' '))>1:print(msg.data.split(' ')[-1]) 

In [ ]:

res=get_keywords_speech(10)
res

In [21]:
drinks=['coke','juice','beer', 'water', 'soda', 'wine']
names=['pete','ana','jack', 'michael', ' my name is' , 'i am','george','mary','ruben','oscar','yolo','mitzi']
confirmation=['yes','no']
gram=drinks+names+confirmation

set_grammar(gram)  ##PRESET DRINKS


result: True

[ WARN] [1685050237.586846998, 3949.832000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3949.861000 according to authority /pose_integrator
[ WARN] [1685050237.586943554, 3949.832000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3949.861000 according to authority /pose_integrator
[ WARN] [1685050237.587011922, 3949.832000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3949.861000 according to authority /pose_integrator
[ WARN] [1685050237.587068643, 3949.832000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3949.861000 according to authority /pose_integrator
[ WARN] [1685050237.587133960, 3949.832000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3949.861000 according to authority /pose_integrator
[ WARN] [1685050237.587191706, 3949.832000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1685050198.172174284, 3932.024000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3932.024000 according to authority /pose_integrator
[ WARN] [1685050198.172267591, 3932.024000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3932.024000 according to authority /pose_integrator
[ WARN] [1685050198.172333731, 3932.024000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3932.024000 according to authority /pose_integrator
[ WARN] [1685050198.172383865, 3932.024000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3932.024000 according to authority unknown_publisher
[ WARN] [1685050198.174050322, 3932.025000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3932.024000 according to authority /pose_integrator
[ WARN] [1685050198.174180814, 3932.025000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[INFO] [1685050202.285918, 3933.928000]: timeout


'timeout'

[ WARN] [1685050205.388006183, 3935.318000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3935.346000 according to authority unknown_publisher
[ WARN] [1685050205.388453513, 3935.318000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3935.346000 according to authority /pose_integrator
[ WARN] [1685050205.388519459, 3935.318000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3935.346000 according to authority /pose_integrator
[ WARN] [1685050205.388579875, 3935.318000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3935.346000 according to authority /pose_integrator
[ WARN] [1685050205.388991756, 3935.318000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3935.346000 according to authority /pose_integrator
[ WARN] [1685050205.389052427, 3935.318000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1685050226.807830002, 3944.873000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3944.911000 according to authority unknown_publisher
[ WARN] [1685050226.809006059, 3944.873000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3944.911000 according to authority /pose_integrator
[ WARN] [1685050226.809097361, 3944.873000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3944.911000 according to authority /pose_integrator
[ WARN] [1685050226.809159172, 3944.873000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3944.911000 according to authority /pose_integrator
[ WARN] [1685050226.809211963, 3944.873000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3944.911000 according to authority /pose_integrator
[ WARN] [1685050226.809267407, 3944.873000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [40]:
res2 = get_keywords_speech(10)
if res2 in['yes','jack','juice']:print('chido',res2)
else: print ('chale',res2)

[ WARN] [1685050553.876236991, 4092.666000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4092.699000 according to authority unknown_publisher
[ WARN] [1685050553.876485156, 4092.666000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4092.699000 according to authority /pose_integrator
[ WARN] [1685050553.876574708, 4092.666000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4092.699000 according to authority /pose_integrator
[ WARN] [1685050553.876622260, 4092.666000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4092.699000 according to authority /pose_integrator
[ WARN] [1685050553.876666507, 4092.666000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4092.699000 according to authority /pose_integrator
[ WARN] [1685050553.876707709, 4092.666000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

chale yes yes


[ WARN] [1685050556.518670252, 4093.872000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4093.889000 according to authority unknown_publisher
[ WARN] [1685050556.519712332, 4093.889000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4093.889000 according to authority /pose_integrator
[ WARN] [1685050556.519788589, 4093.889000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4093.889000 according to authority /pose_integrator
[ WARN] [1685050556.519843782, 4093.889000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4093.889000 according to authority /pose_integrator
[ WARN] [1685050556.519891648, 4093.889000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4093.889000 according to authority /pose_integrator
[ WARN] [1685050556.519939014, 4093.889000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1685050568.609608027, 4099.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4099.271000 according to authority unknown_publisher
[ WARN] [1685050568.610695566, 4099.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4099.271000 according to authority /pose_integrator
[ WARN] [1685050568.610761097, 4099.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4099.271000 according to authority /pose_integrator
[ WARN] [1685050568.610814349, 4099.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4099.271000 according to authority /pose_integrator
[ WARN] [1685050568.610862161, 4099.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4099.271000 according to authority /pose_integrator
[ WARN] [1685050568.610909799, 4099.228000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1685050579.639557202, 4104.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4104.532000 according to authority /pose_integrator
[ WARN] [1685050579.639644687, 4104.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4104.532000 according to authority /pose_integrator
[ WARN] [1685050579.639692925, 4104.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4104.532000 according to authority unknown_publisher
[ WARN] [1685050579.639740853, 4104.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4104.532000 according to authority /pose_integrator
[ WARN] [1685050579.639794053, 4104.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4104.532000 according to authority /pose_integrator
[ WARN] [1685050579.639837768, 4104.496000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

...

[ WARN] [1685053025.099250427, 5216.588000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5216.615000 according to authority /pose_integrator
[ WARN] [1685053025.099341593, 5216.588000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5216.615000 according to authority /pose_integrator
[ WARN] [1685053025.099412871, 5216.588000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5216.615000 according to authority /pose_integrator
[ WARN] [1685053025.099462252, 5216.588000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5216.615000 according to authority /pose_integrator
[ WARN] [1685053025.099514382, 5216.588000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5216.615000 according to authority /pose_integrator
[ WARN] [1685053025.099552289, 5216.588000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1685053038.488117389, 5222.723000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5222.723000 according to authority unknown_publisher
[ WARN] [1685053038.495094044, 5222.723000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5222.723000 according to authority /pose_integrator
[ WARN] [1685053038.495178366, 5222.723000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5222.723000 according to authority /pose_integrator
[ WARN] [1685053038.495234976, 5222.723000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5222.723000 according to authority /pose_integrator
[ WARN] [1685053038.495304210, 5222.723000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5222.723000 according to authority /pose_integrator
[ WARN] [1685053038.495356415, 5222.723000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1685053055.880628925, 5230.743000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5230.743000 according to authority unknown_publisher
[ WARN] [1685053055.884280515, 5230.743000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5230.743000 according to authority /pose_integrator
[ WARN] [1685053055.884367194, 5230.743000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5230.743000 according to authority /pose_integrator
[ WARN] [1685053055.884418050, 5230.743000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5230.743000 according to authority /pose_integrator
[ WARN] [1685053055.884458729, 5230.743000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5230.743000 according to authority /pose_integrator
[ WARN] [1685053055.884505517, 5230.743000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1685053067.696594091, 5236.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5236.137000 according to authority /pose_integrator
[ WARN] [1685053067.696662724, 5236.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5236.137000 according to authority /pose_integrator
[ WARN] [1685053067.696718296, 5236.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5236.137000 according to authority /pose_integrator
[ WARN] [1685053067.696762646, 5236.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5236.137000 according to authority /pose_integrator
[ WARN] [1685053067.696812513, 5236.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5236.137000 according to authority /pose_integrator
[ WARN] [1685053067.696852763, 5236.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1685053078.992583783, 5241.349000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5241.392000 according to authority unknown_publisher
[ WARN] [1685053078.992799603, 5241.349000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5241.392000 according to authority /pose_integrator
[ WARN] [1685053078.992858198, 5241.349000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5241.392000 according to authority /pose_integrator
[ WARN] [1685053078.992909103, 5241.349000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5241.392000 according to authority /pose_integrator
[ WARN] [1685053078.992956146, 5241.349000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 5241.392000 according to authority /pose_integrator
[ WARN] [1685053078.993002212, 5241.349000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [25]:
set_grammar(['yes','no'])

result: True

[ WARN] [1685050309.054422459, 3982.526000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3982.562000 according to authority unknown_publisher
[ WARN] [1685050309.054500893, 3982.526000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3982.562000 according to authority /pose_integrator
[ WARN] [1685050309.054560646, 3982.526000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3982.562000 according to authority /pose_integrator
[ WARN] [1685050309.054596850, 3982.526000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3982.562000 according to authority /pose_integrator
[ WARN] [1685050309.054636793, 3982.526000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 3982.562000 according to authority /pose_integrator
[ WARN] [1685050309.054685753, 3982.526000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [38]:
res2 in ['yes', 'no']

False

[ WARN] [1685050514.677100188, 4075.180000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4075.203000 according to authority /pose_integrator
[ WARN] [1685050514.677185321, 4075.180000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4075.203000 according to authority /pose_integrator
[ WARN] [1685050514.677234582, 4075.180000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4075.203000 according to authority /pose_integrator
[ WARN] [1685050514.677275514, 4075.180000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4075.203000 according to authority /pose_integrator
[ WARN] [1685050514.677313634, 4075.180000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4075.203000 according to authority /pose_integrator
[ WARN] [1685050514.677354143, 4075.180000000]: TF_REPEATED_DATA ignoring data with redundant timestamp f

[ WARN] [1685050538.105894590, 4085.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4085.765000 according to authority unknown_publisher
[ WARN] [1685050538.106015338, 4085.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4085.765000 according to authority /pose_integrator
[ WARN] [1685050538.106084311, 4085.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4085.765000 according to authority /pose_integrator
[ WARN] [1685050538.106162102, 4085.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4085.765000 according to authority /pose_integrator
[ WARN] [1685050538.106216074, 4085.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4085.765000 according to authority /pose_integrator
[ WARN] [1685050538.106257591, 4085.737000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [37]:
res2='not'

[ WARN] [1685050513.651198520, 4074.687000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4074.735000 according to authority unknown_publisher
[ WARN] [1685050513.651914672, 4074.687000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4074.735000 according to authority /pose_integrator
[ WARN] [1685050513.651971023, 4074.687000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4074.735000 according to authority /pose_integrator
[ WARN] [1685050513.652017069, 4074.687000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4074.735000 according to authority /pose_integrator
[ WARN] [1685050513.652066294, 4074.687000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 4074.735000 according to authority /pose_integrator
[ WARN] [1685050513.652113464, 4074.687000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 